In [ ]:
!pip install ultralytics

In [ ]:
!pip install python-doctr

In [ ]:
import imageio
import time
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import re
import cv2

# Загружаем модель
yolo = YOLO('best_detect.pt')

#Загружаем OCR модель
model = ocr_predictor(pretrained=True)


# ф-я для нахождения номера на картинке
def found_licence(img: str) -> bool:
  image = cv2.imread(img)
  results = yolo(img, conf=0.5)
  if results[0].boxes.xyxy.cpu().shape[0] == 0:
    return False
  #results[0].show()

  cor = results[0].boxes.xyxy.cpu().numpy().round()[0].astype(int)
  x1, y1, x2, y2 = cor[0], cor[1], cor[2], cor[3]
  cropped = image[y1:y2, x1:x2]
  #cv2_imshow(cropped)
  cv2.imwrite('output.jpeg', cropped)
  #return read_licence('output.jpeg')
  return True


# ф-я для чтения номера: сначала конвертируем в черно-белое, затем читаем
def read_licence(img: str) -> str:
  image = cv2.imread(img)
  height, width, _ = image.shape
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
  cv2_imshow(thresh)

  cv2.imwrite('output.jpeg', thresh)
  doc = DocumentFile.from_images('output.jpeg')
  result = model(doc)
  full_text = ""
  russian_letters_digits_pattern = re.compile(r'^[ABEKMHOPCTУХ-ЯЁ0-9]+$')
  only_digits_pattern = re.compile(r'^[0-9]+$')

  for page in result.pages:
      for block in page.blocks:
          for line in block.lines:
              line_words = []
              for word in line.words:
                  word_text = word.value
                  if (russian_letters_digits_pattern.match(word_text) or
                      only_digits_pattern.match(word_text)):
                      line_words.append(word_text)
              if line_words:
                  for i in line_words:
                    full_text += i

  full_text = ''.join(sorted(full_text.split(), key=len, reverse=True))
  return full_text


#НАРЕЗАЕМ ВИДЕО НА КАДРЫ
reader = imageio.get_reader('/content/IMG_7850.MOV')
for frame_number, im in enumerate(reader):
  if frame_number % 1 == 0:
    cv2.imwrite('output.jpeg', im)
    a = found_licence('output.jpeg')
    if a:
      lic = read_licence('output.jpeg') # номер
      print(lic)